![CMCC](http://cmcc.ufabc.edu.br/images/logo_site.jpg)
    

#  Projeto Final - Inteligência na Web e Big Data 


###### Aluna: Angélica Custódio

#### Esse arquivo tem a sequência de comandos utilizados para implementar o algoritmo do Teste Kolmogorov-Smirnov com o conceito de dados distribuídos, com base nos aprendizados adiquiridos ao longo da disciplina, utilizando o Spark através de sua interface com a linguagem Python.

In [1]:
sc = SparkContext.getOrCreate()

#### 1ª Parte

#### Nessa parte vamos importar uma base de dados fictícia que possue as colunas ID, Score e Perf, respectivamente. Essa base é uma simulação de um contexto de Credit Scoring onde ID é a identificação de cada indivíduo, Score é a pontuação proveniente da Regressão Logística (Probabilidade*1000) e a Perf é uma marcação de clientes bons e maus tal que: 
#### Perf = 1, então cliente se tornou inadimplente;
#### Perf = 0, então cliente não se tornou inadimplente.

In [2]:
baseDir = os.path.join('C:\Users\igor.margen\Documents')
inputPath = os.path.join('Projeto', 'Dados_projeto.txt')
fileName = os.path.join(baseDir, inputPath)

print "MYPATH:", fileName

numPartitions = 3
dados = sc.textFile(fileName, numPartitions)

dados.collect()

MYPATH: C:\Users\igor.margen\Documents\Projeto\Dados_projeto.txt


[u'1,287,0',
 u'2,877,1',
 u'3,908,1',
 u'4,991,0',
 u'5,52,0',
 u'6,183,1',
 u'7,171,1',
 u'8,784,0',
 u'9,170,1',
 u'10,189,1',
 u'11,284,0',
 u'12,368,0',
 u'13,120,1',
 u'14,548,0',
 u'15,97,1',
 u'16,152,1',
 u'17,918,0',
 u'18,217,1',
 u'19,911,1',
 u'20,356,0',
 u'21,99,1',
 u'22,912,1',
 u'23,550,0',
 u'24,105,1',
 u'25,877,0',
 u'26,911,0',
 u'27,864,1',
 u'28,941,1',
 u'29,962,0',
 u'30,3,1',
 u'31,350,1',
 u'32,174,0',
 u'33,729,1',
 u'34,119,0',
 u'35,210,0',
 u'36,234,1',
 u'37,484,0',
 u'38,288,1',
 u'39,955,0',
 u'40,373,0',
 u'41,445,1',
 u'42,662,0',
 u'43,78,1',
 u'44,336,1',
 u'45,850,0',
 u'46,558,0',
 u'47,615,0',
 u'48,242,0',
 u'49,619,1',
 u'50,195,0',
 u'51,27,0',
 u'52,503,0',
 u'53,724,1',
 u'54,927,1',
 u'55,834,0',
 u'56,801,1',
 u'57,356,0',
 u'58,139,0',
 u'59,434,0',
 u'60,807,0',
 u'61,559,0',
 u'62,129,1',
 u'63,904,1',
 u'64,720,1',
 u'65,917,0',
 u'66,149,0',
 u'67,58,1',
 u'68,109,1',
 u'69,135,1',
 u'70,427,0',
 u'71,235,0',
 u'72,738,1',
 u'73,709

In [3]:
numobs = dados.count()
print numobs #numeros de observações
amostra5 = dados.take(5)
print amostra5 #exemplo das 5 primeiras observações

200
[u'1,287,0', u'2,877,1', u'3,908,1', u'4,991,0', u'5,52,0']


#### A bases de dados utilizada é rotulada, portanto, armazenaremos usando o objeto LabeledPoint. 

In [4]:
from pyspark.mllib.regression import LabeledPoint
import numpy as np

def parsePoint(line):
    Point = line.split(',')
    return LabeledPoint(Point[0],Point[1:] )

parsedPoints = map(parsePoint,amostra5)
primeiroScore = parsedPoints[0].features
primeiroTarget = parsedPoints[0].label
print primeiroScore, primeiroTarget

[287.0,0.0] 1.0


#### Avaliando a dimensão do Score

In [5]:
#Aplicando a função de LabeledPoint no conjunto de dados completo
dados2= dados.map(parsePoint)

#Selecionando apenas a coluna do Score
dadosScore = dados2.map(lambda x: x.features[0])

#Calculando as dimensões mínima e máxima
minScore = dadosScore.min()
maxScore = dadosScore.max()
print maxScore, minScore

991.0 3.0


#### 2ª Parte

#### Já sabemos que os nossos valores de Score vão de 3 a 991, agora vamos criar os percentis utilizando a função retornaPercentil() que construimos a seguir.
#### Depois vamos aplicar a transformação em cada palavra do RDD.

In [7]:
#preparando a base para calculo dos percentis
dadosPrep= (dados2
             .map(lambda x: x.features[0])
             .sortBy(lambda x: x)
             .zipWithIndex()
             .map(lambda x: (x[1],x[0]))
            )

In [8]:
#criando a função que calcula percentis de referência
def retornaPercentil(valores, percentile):
    assert percentile > 0 and percentile <= 100, "valores de percentil válidos entre 0 até 100"

    return valores.lookup(np.ceil(valores.count() / 100 * percentile - 1))[0]

perc10=retornaPercentil(dadosPrep,10)
perc20=retornaPercentil(dadosPrep,20)
perc30=retornaPercentil(dadosPrep,30)
perc40=retornaPercentil(dadosPrep,40)
perc50=retornaPercentil(dadosPrep,50)
perc60=retornaPercentil(dadosPrep,60)
perc70=retornaPercentil(dadosPrep,70)
perc80=retornaPercentil(dadosPrep,80)
perc90=retornaPercentil(dadosPrep,90)
perc100=retornaPercentil(dadosPrep,100)

#### Agora que temos os percentis de referência calculados classificaremos cada um dos Scores pontuais da nossa base

In [9]:
#Criando a função que classifca os Score pontuais em cada um dos 10 percentis
def percentilScore(score):
        if   score < perc10: return (score,1)
        elif score < perc20: return (score,2)
        elif score < perc30: return (score,3)
        elif score < perc40: return (score,4)
        elif score < perc50: return (score,5)
        elif score < perc60: return (score,6)
        elif score < perc70: return (score,7)
        elif score < perc80: return (score,8)
        elif score < perc90: return (score,9)
        else: return (score,10)

In [13]:
def aplica(data):
    return(data.features[1], percentilScore(data.features[0]))

dadosnovo=dados2.map(aplica)
dadosnovo.collect()

[(0.0, (287.0, 3)),
 (1.0, (877.0, 9)),
 (1.0, (908.0, 10)),
 (0.0, (991.0, 10)),
 (0.0, (52.0, 1)),
 (1.0, (183.0, 2)),
 (1.0, (171.0, 2)),
 (0.0, (784.0, 8)),
 (1.0, (170.0, 2)),
 (1.0, (189.0, 2)),
 (0.0, (284.0, 3)),
 (0.0, (368.0, 4)),
 (1.0, (120.0, 2)),
 (0.0, (548.0, 6)),
 (1.0, (97.0, 1)),
 (1.0, (152.0, 2)),
 (0.0, (918.0, 10)),
 (1.0, (217.0, 3)),
 (1.0, (911.0, 10)),
 (0.0, (356.0, 4)),
 (1.0, (99.0, 1)),
 (1.0, (912.0, 10)),
 (0.0, (550.0, 6)),
 (1.0, (105.0, 1)),
 (0.0, (877.0, 9)),
 (0.0, (911.0, 10)),
 (1.0, (864.0, 9)),
 (1.0, (941.0, 10)),
 (0.0, (962.0, 10)),
 (1.0, (3.0, 1)),
 (1.0, (350.0, 4)),
 (0.0, (174.0, 2)),
 (1.0, (729.0, 8)),
 (0.0, (119.0, 2)),
 (0.0, (210.0, 3)),
 (1.0, (234.0, 3)),
 (0.0, (484.0, 5)),
 (1.0, (288.0, 3)),
 (0.0, (955.0, 10)),
 (0.0, (373.0, 4)),
 (1.0, (445.0, 5)),
 (0.0, (662.0, 7)),
 (1.0, (78.0, 1)),
 (1.0, (336.0, 4)),
 (0.0, (850.0, 9)),
 (0.0, (558.0, 6)),
 (0.0, (615.0, 6)),
 (0.0, (242.0, 3)),
 (1.0, (619.0, 6)),
 (0.0, (195.0, 3)

In [26]:
#Criando função que conte a frequência de Bons e Maus por conjunto de Percentis
from itertools import groupby
    
def freq(conj):
    for i in range(1,10):
        [len(list(conj.features[1]==i)) for key, group in groupby(conj.features[1]==i)]

teste2=dadosnovo.map(freq)

### Os próximos passos seriam:
#### 1) Criar a contagem acumulada de bons e maus em cada categoria
#### 2) Calcular a razão da frequencia acumulada em relação ao total de bons e maus
#### 3) Calcular as diferenças entre cada uma das frequencias relativas obtidas no passo anterior
#### 4) Computar a máxima diferença dentre as diferenças calculadas